## NLP Assignment-2
### Mohammed Alom
### Student Number R00144214
### Task-1 Chatbot with Twitter Chatlog.
##### Reference - Dataset and idea and code was collected from
This code and ideas is taken from an open source different github repos and been tweeked in the code to meet the requirements.
 

#### Readme
#### How to Run:

1. To start chatting please goto Menu ->Runtime-> Run all or directly press (ctrl + F9) from keyboard.
2. intents.json file need to upload to continue chat
 

In [67]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [68]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU {}'.format(tpu.cluster_spec().as_dict()['worker']))
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: {}".format(strategy.num_replicas_in_sync))

REPLICAS: 1


In [69]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [70]:
import sys
import os
#os.chdir("/content/drive/My Drive/Colab Notebooks/chatbot/")
#os.listdir()
#%cd /content/drive/My Drive/Colab Notebooks/

#py_file_location = "/content/drive/My Drive/Colab Notebooks/intents.json/"
#sys.path.append(os.path.abspath(py_file_location))


#import intents

In [71]:
#datasetDir = '/content/drive/My Drive/Colab Notebooks/intents.json'

Here intents.json used only for the chat session. so, once the colab is terminate the intents.json file will be not available.

In [72]:
with open('intents.json') as file:
    data = json.load(file)

In [73]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [74]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

num_classes = len(labels)

In [75]:
vocab_size = 10000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [76]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [77]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 16)            160000    
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 8)                 136       
Total params: 160,680
Trainable params: 160,680
Non-trainable params: 0
_________________________________________________________________


In [78]:
EPOCHS = 500
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(padded_sequences, np.array(training_labels), epochs=EPOCHS)

Epoch 1/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0804 - accuracy: 0.0919
Epoch 2/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0785 - accuracy: 0.0919
Epoch 3/500
2/2 [==============================] - 0s 5ms/step - loss: 2.0768 - accuracy: 0.0919
Epoch 4/500
2/2 [==============================] - 0s 6ms/step - loss: 2.0752 - accuracy: 0.0919
Epoch 5/500
2/2 [==============================] - 0s 5ms/step - loss: 2.0745 - accuracy: 0.2652
Epoch 6/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0733 - accuracy: 0.2143
Epoch 7/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0717 - accuracy: 0.2143
Epoch 8/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0711 - accuracy: 0.2039
Epoch 9/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0694 - accuracy: 0.2143
Epoch 10/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0693 - accuracy: 0.2039
Epoch 11/500
2/2 [==========

In [79]:
model.save("chat_model")

import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


In [80]:
def check():
    print('Start talking with bot, Enter quit to exit from the chat.')
    while True:
        string = input('Human : ')
        if string == 'quit': break
        result = model.predict(pad_sequences(tokenizer.texts_to_sequences([string]),
                                             truncating='post', maxlen=max_len))
        category = lbl_encoder.inverse_transform([np.argmax(result)]) # labels[np.argmax(result)]
        for i in data['intents']:
            if i['tag']==category:
                print(np.random.choice(i['responses']))
    print('Thank you for chat.')

In [81]:
check()

Start talking with bot, Enter quit to exit from the chat.
Human : Hi
Hi there
Human : How are you today?
Hello
Human : What is your name?
Just call me as Joana
Human : Nice to talk to you.
Any time!
Human : Bye Bye..
Bye! Come back again
Human : quit
Thank you for chat.


#### Summary

It will be more effective If we put more intents with different category.  